In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# =====================================================
# STEP 1 — LOAD DATA
# =====================================================

train_path = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/train.csv"
test_path  = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/test.csv"



df_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

TARGET = "spend_category"
ID_COL = "trip_id"

print("Original Training Shape:", df_raw.shape)
print("Original Test Shape:", test_raw.shape)


# =====================================================
# HELPER — Convert Range (15-30, 90+) to numeric
# =====================================================
def range_to_mid(x):
    x = str(x).strip()
    if x.lower() in ["none", "", "nan", "null"]:
        return np.nan
    if "+" in x:
        return float(x.replace("+", ""))
    if "-" in x:
        a, b = x.split("-")
        return (float(a) + float(b)) / 2
    try:
        return float(x)
    except:
        return np.nan


# =====================================================
# STEP 2 — GLOBAL COLUMN DEFINITIONS
# =====================================================
binary_cols = [
    "is_first_visit","intl_transport_included","accomodation_included",
    "food_included","domestic_transport_included","sightseeing_included",
    "guide_included","insurance_included"
]

categorical_cols = [
    "country","age_group","travel_companions","main_activity",
    "visit_purpose","tour_type","info_source","arrival_weather"
]

numeric_count_cols = ["num_females","num_males","mainland_stay_nights","island_stay_nights"]


# =====================================================
# STEP 3 — REMOVE NULL TARGETS FIRST
# =====================================================
removed_target_nulls = df_raw[TARGET].isnull().sum()
print("Rows removed due to null spend_category:", removed_target_nulls)

df_raw = df_raw[df_raw[TARGET].notnull()].reset_index(drop=True)
print("Training shape after removing null targets:", df_raw.shape)


# =====================================================
# STEP 4 — MAIN PREPROCESSING FUNCTION
# =====================================================
def preprocess_raw_df(df):
    df = df.copy()

    # Clean strings
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.rstrip(',')

    # Binary processing
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
            df[c] = df[c].replace({
                "yes": 1,
                "no": 0,
                "nan": np.nan,
                "none": np.nan,
                "null": np.nan,
                "": np.nan
            })
            df[c] = df[c].fillna(0).astype(int)

    # Numeric count fields
    for c in numeric_count_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # =====================================================
    # SAFE ORDINAL ENCODING FOR RANGE COLUMNS
    # =====================================================

    # Clean weird string values
    def clean_str(x):
        x = str(x).strip().lower()
        if x in ["nan", "none", "null", ""]:
            return np.nan
        return x

    # Clean raw string columns
    if "days_booked_before_trip" in df.columns:
        df["days_booked_before_trip_clean"] = df["days_booked_before_trip"].apply(clean_str)

    if "total_trip_days" in df.columns:
        df["total_trip_days_clean"] = df["total_trip_days"].apply(clean_str)

    # Define ordinal mappings
    ordinal_days_booked = {
        "1-7": 1,
        "8-14": 2,
        "15-30": 3,
        "31-60": 4,
        "61-90": 5,
        "90+": 6
    }

    ordinal_total_trip = {
        "1-6": 1,
        "7-14": 2,
        "15-30": 3,
        "30+": 4
    }

    # Map → Fill Missing → Convert to int
    if "days_booked_before_trip_clean" in df.columns:
        df["days_booked_before_trip_ord"] = (
            df["days_booked_before_trip_clean"]
                .map(ordinal_days_booked)
        )

        # Fill NaN with mode **of ordinal values**
        df["days_booked_before_trip_ord"].fillna(
            df["days_booked_before_trip_ord"].mode()[0],
            inplace=True
        )

        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_ord"].astype(int)

    if "total_trip_days_clean" in df.columns:
        df["total_trip_days_ord"] = (
            df["total_trip_days_clean"]
                .map(ordinal_total_trip)
        )

        # Fill NaN with mode of ordinal values
        df["total_trip_days_ord"].fillna(
            df["total_trip_days_ord"].mode()[0],
            inplace=True
        )

        df["total_trip_days_ord"] = df["total_trip_days_ord"].astype(int)



    # Special requirements → binary
    if "has_special_requirements" in df.columns:
        df["has_special_req_bin"] = df["has_special_requirements"].astype(str).apply(
            lambda x: 0 if x.lower() in ["none", "", "nan"] else 1
        )

    return df


# =====================================================
# APPLY PREPROCESSING TO TRAIN & TEST
# =====================================================
df = preprocess_raw_df(df_raw).reset_index(drop=True)
test_df = preprocess_raw_df(test_raw).reset_index(drop=True)

print("\nAfter Base Preprocessing:")
print(df.shape)


# =====================================================
# STEP 5 — IMPUTATIONS (NO NaNs must remain)
# =====================================================

# Categorical mode fill
for c in categorical_cols:
    if c in df.columns:
        mode = df[c].mode()[0]
        df[c] = df[c].fillna(mode)
        test_df[c] = test_df[c].fillna(mode)

# =========================================================
# OUTLIER REMOVAL (TRAIN ONLY)
# And count number of rows removed per condition
# =========================================================

clean_df = df.copy()
initial_rows = len(clean_df)

outlier_info = {}

# ----- num_females ≤ 14 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_females"] <= 10]
after = len(clean_df)
outlier_info["num_females"] = before - after

# ----- Rule 1: num_males ≤ 13 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_males"] <= 10]
after_rule1 = len(clean_df)
removed_rule1 = before - after_rule1
before_rule2 = len(clean_df)

# Save results
outlier_info["num_males_threshold"] = removed_rule1
print("Removed (num_males > 20):", removed_rule1)


# ----- mainland_stay_nights ≤ 100 -----
before = len(clean_df)
clean_df = clean_df[clean_df["mainland_stay_nights"] <= 90]
after = len(clean_df)
outlier_info["mainland_stay_nights"] = before - after

# ----- island_stay_nights ≤ 21 -----
before = len(clean_df)
clean_df = clean_df[clean_df["island_stay_nights"] <= 60]
after = len(clean_df)
outlier_info["island_stay_nights"] = before - after

final_rows = len(clean_df)

# =========================================================
# PRINT OUTLIER REMOVAL SUMMARY
# =========================================================
print("\n========== OUTLIER REMOVAL SUMMARY ==========")
for col, removed in outlier_info.items():
    print(f"{col}: removed {removed} rows")

print("---------------------------------------------")
print(f"Total rows removed: {initial_rows - final_rows}")
print(f"Final Training Shape after Outlier Removal: {clean_df.shape}")
print("Test Shape (unchanged):", test_df.shape)


# =====================================================
# STEP 5 — FINAL FEATURE DEFINITIONS (No NaNs left)
# =====================================================

numeric_features = [
    "num_females",
    "num_males",
    "mainland_stay_nights",
    "island_stay_nights",
    "days_booked_before_trip_ord",
    "total_trip_days_ord"
]

binary_features = binary_cols + ["has_special_req_bin"]

categorical_features = categorical_cols

all_features = numeric_features + binary_features + categorical_features

clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df[all_features]




Original Training Shape: (12654, 25)
Original Test Shape: (5852, 24)
Rows removed due to null spend_category: 34
Training shape after removing null targets: (12620, 25)


/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] =


After Base Preprocessing:
(12620, 30)
Removed (num_males > 20): 8

========== OUTLIER REMOVAL SUMMARY ==========
num_females: removed 28 rows
num_males_threshold: removed 8 rows
mainland_stay_nights: removed 27 rows
island_stay_nights: removed 8 rows
---------------------------------------------
Total rows removed: 71
Final Training Shape after Outlier Removal: (12549, 30)
Test Shape (unchanged): (5852, 29)


In [2]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# =====================================================
# STEP X — FEATURE ENGINEERING: K-Means + GMM CLUSTERS
# =====================================================

print("\n=== Running Clustering Feature Engineering (KMeans) ===")

# ----------------------------
# 1. Prepare numeric data only
# ----------------------------
num_only = clean_df[numeric_features].copy()
test_num_only = test_df_final[numeric_features].copy()

# Standardize for clustering
scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_only)
test_num_scaled = scaler.transform(test_num_only)

# ----------------------------
# 2. K-Means Clustering
# ----------------------------
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
clean_df["kmeans_cluster"] = kmeans.fit_predict(num_scaled)
test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)

# # ----------------------------
# # 3. Gaussian Mixture Model (GMM)
# # ----------------------------
# gmm = GaussianMixture(n_components=5, random_state=42)
# clean_df["gmm_cluster"] = gmm.fit_predict(num_scaled)
# test_df_final["gmm_cluster"] = gmm.predict(test_num_scaled)

print("Clustering features added: kmeans_cluster")

# Add to categorical features
categorical_features += ["kmeans_cluster"]

# Update full feature list
all_features = numeric_features + binary_features + categorical_features

# Important: keep only final columns
clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df_final[all_features]

print("\nFinal shape after clustering:", clean_df.shape)



=== Running Clustering Feature Engineering (KMeans) ===
Clustering features added: kmeans_cluster

Final shape after clustering: (12549, 25)


/tmp/ipython-input-1854590008.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
    ("bin", "passthrough", binary_features)
])


In [4]:
# =====================================================
# CORRECT SPLITTING ACCORDING TO YOUR REQUIREMENT
# =====================================================

from sklearn.model_selection import train_test_split

# ----------- FULL DATASET -----------
full_df = clean_df.copy()

# =====================================================
# SPLIT A: 80% train, 10% val, 10% test
# =====================================================

# First: 80% train, 20% temp
train_80, temp_20 = train_test_split(
    full_df,
    test_size=0.20,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 20% temp → 10% val + 10% test
val_10, test_10 = train_test_split(
    temp_20,
    test_size=0.50,   # half of 20%
    stratify=temp_20[TARGET],
    random_state=42
)

print("Split A (80/10/10):")
print("train_80:", train_80.shape)
print("val_10:", val_10.shape)
print("test_10:", test_10.shape)


# =====================================================
# SPLIT B: 20% train, 40% val, 40% test
# =====================================================

# First: 20% train, 80% temp
train_20, temp_80 = train_test_split(
    full_df,
    test_size=0.80,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 80% temp → 40% val + 40% test
val_40, test_40 = train_test_split(
    temp_80,
    test_size=0.50,
    stratify=temp_80[TARGET],
    random_state=42
)

print("\nSplit B (20/40/40):")
print("train_20:", train_20.shape)
print("val_40:", val_40.shape)
print("test_40:", test_40.shape)


Split A (80/10/10):
train_80: (10039, 25)
val_10: (1255, 25)
test_10: (1255, 25)

Split B (20/40/40):
train_20: (2509, 25)
val_40: (5020, 25)
test_40: (5020, 25)


In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
from sklearn.base import clone
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

SEED = 42
np.random.seed(SEED)

# =====================================================
# MLP Architectures (MULTICLASS)
# =====================================================

def build_mlp_small(input_dim, num_classes):
    return keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

def build_mlp_medium(input_dim, num_classes):
    return keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

def build_mlp_large(input_dim, num_classes):
    return keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

MLP_BUILDERS = {
    "MLP_SMALL":  build_mlp_small,
    "MLP_MEDIUM": build_mlp_medium,
    "MLP_LARGE":  build_mlp_large,
}

# NOTE: final Kaggle test features are already in test_df_final[all_features]
# and original IDs are in test_raw[ID_COL] or test_df[ID_COL].

BASE_NN_DIR = "/content/drive/MyDrive/ML_Project/MultinomialClassification/NN/SMOTE"


# =====================================================
# TRAIN ONE MODEL (one size + one split config)
# =====================================================
def train_mlp_smote(model_name,
                    X_train_df, y_train,
                    X_val_df,   y_val,
                    X_test_int_df, y_test_int,
                    run_name):

    # Folder structure:
    # NN/SMOTE/MLP_SMALL/80_10_10/...
    out_dir = os.path.join(BASE_NN_DIR, model_name, run_name)
    os.makedirs(out_dir, exist_ok=True)

    # Clone ColumnTransformer so each model has its own fitted preprocessor
    local_prep = clone(preprocess)
    local_prep.fit(X_train_df)

    X_train = local_prep.transform(X_train_df)
    X_val   = local_prep.transform(X_val_df)
    X_test  = local_prep.transform(X_test_int_df)

    # SMOTE on training set (MULTICLASS OK)
    sm = SMOTE(random_state=SEED)
    X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

    input_dim   = X_train_sm.shape[1]
    num_classes = len(np.unique(y_train_sm))

    # Pick architecture
    builder = MLP_BUILDERS[model_name]
    model = builder(input_dim, num_classes)

    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    es = keras.callbacks.EarlyStopping(
        patience=6,
        monitor="val_loss",
        restore_best_weights=True
    )

    print(f"\n🚀 Training {model_name} — {run_name} ...")

    model.fit(
        X_train_sm, y_train_sm,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=256,
        callbacks=[es],
        verbose=1
    )

    # ------------- VALIDATION -------------
    val_probs = model.predict(X_val)
    val_pred  = np.argmax(val_probs, axis=1)
    val_acc   = accuracy_score(y_val, val_pred)
    val_rep   = classification_report(y_val, val_pred)

    with open(os.path.join(out_dir, "validation_report.txt"), "w") as f:
        f.write(val_rep)
        f.write(f"\nValidation Accuracy: {val_acc}\n")

    # ------------- INTERNAL TEST -------------
    test_probs = model.predict(X_test)
    test_pred  = np.argmax(test_probs, axis=1)
    test_acc   = accuracy_score(y_test_int, test_pred)
    test_rep   = classification_report(y_test_int, test_pred)

    with open(os.path.join(out_dir, "test_report.txt"), "w") as f:
        f.write(test_rep)
        f.write(f"\nTest Accuracy: {test_acc}\n")

    # ------------- SUMMARY FILE -------------
    with open(os.path.join(out_dir, "summary.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_acc}\n")
        f.write(f"Test Accuracy: {test_acc}\n")

    # ------------- FINAL KAGGLE SUBMISSION -------------
    X_final = local_prep.transform(test_df_final[all_features])
    final_probs = model.predict(X_final)
    final_pred  = np.argmax(final_probs, axis=1)

    submission = pd.DataFrame({
        ID_COL: test_raw[ID_COL],     # trip_id
        TARGET: final_pred            # spend_category
    })
    submission.to_csv(
        os.path.join(out_dir, f"{model_name}_{run_name}_submission.csv"),
        index=False
    )

    print(f"[DONE] {model_name} — {run_name} ✓\n")


# =====================================================
# RUN ALL MLP SIZES ON BOTH SPLITS
# =====================================================
def run_all_mlps_smote():

    # ---- Split A: 80 / 10 / 10 ----
    X80   = train_80[all_features]
    y80   = train_80[TARGET].values

    Xv10  = val_10[all_features]
    yv10  = val_10[TARGET].values

    Xt10  = test_10[all_features]
    yt10  = test_10[TARGET].values

    # ---- Split B: 20 / 40 / 40 ----
    X20   = train_20[all_features]
    y20   = train_20[TARGET].values

    Xv40  = val_40[all_features]
    yv40  = val_40[TARGET].values

    Xt40  = test_40[all_features]
    yt40  = test_40[TARGET].values

    for model_name in ["MLP_SMALL", "MLP_MEDIUM", "MLP_LARGE"]:
        print(f"\n============================")
        print(f" Running: {model_name}")
        print(f"============================\n")

        # 80 / 10 / 10
        train_mlp_smote(
            model_name,
            X80, y80,
            Xv10, yv10,
            Xt10, yt10,
            run_name="80_10_10"
        )

        # 20 / 40 / 40
        train_mlp_smote(
            model_name,
            X20, y20,
            Xv40, yv40,
            Xt40, yt40,
            run_name="20_40_40"
        )

    print("\n✅ All SMOTE+MLP runs completed.\n")


# Actually run everything
run_all_mlps_smote()



 Running: MLP_SMALL


🚀 Training MLP_SMALL — 80_10_10 ...
Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5171 - loss: 0.9637 - val_accuracy: 0.6693 - val_loss: 0.7090
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6963 - loss: 0.7080 - val_accuracy: 0.6972 - val_loss: 0.6692
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7164 - loss: 0.6611 - val_accuracy: 0.7195 - val_loss: 0.6537
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7262 - loss: 0.6395 - val_accuracy: 0.7124 - val_loss: 0.6475
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7306 - loss: 0.6296 - val_accuracy: 0.7267 - val_loss: 0.6315
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7324 - loss: 0.6223 - val_accuracy: 0.7179 - val_loss: 0.6490
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7434 - loss: 0.6059 - val_accuracy: 0.7195 - val_loss: 0.6600
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accurac